In [1]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

# from sklearn.linear_model import LinearRegression ## likes, dislikes 대체
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

import os
from dotenv import load_dotenv

In [2]:
file_path = 'C:/py_src/awake/data/'

In [3]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv(file_path + 'merge_df_users_fin_eda.csv', low_memory=False)
# youtube_videos = pd.read_csv(file_path + 'youtube_videos_eda.csv')

## 계정 데이터 분석

In [4]:
# 불필요정보 제거 - y값 제거(다른 모델)
merge_df_users_fin = merge_df_users_fin.drop('y_label',axis=1)
## y값 : merge_df_users_fin['subscribers_count']

### 기간별 피처 생성

In [5]:
# 주별, 월별, 분기별 변수 생성

# 구독자 관련 변수
merge_df_users_fin['weekly_subscribers_gained'] = merge_df_users_fin['subscribers_gained'].rolling(window=7).sum()
merge_df_users_fin['monthly_subscribers_gained'] = merge_df_users_fin['subscribers_gained'].rolling(window=30).sum()
merge_df_users_fin['quarterly_subscribers_gained'] = merge_df_users_fin['subscribers_gained'].rolling(window=90).sum()

merge_df_users_fin['weekly_subscribers_lost'] = merge_df_users_fin['subscribers_lost'].rolling(window=7).sum()
merge_df_users_fin['monthly_subscribers_lost'] = merge_df_users_fin['subscribers_lost'].rolling(window=30).sum()
merge_df_users_fin['quarterly_subscribers_lost'] = merge_df_users_fin['subscribers_lost'].rolling(window=90).sum()

merge_df_users_fin['weekly_net_subscribers_change'] = merge_df_users_fin['weekly_subscribers_gained'] - merge_df_users_fin['weekly_subscribers_lost']
merge_df_users_fin['monthly_net_subscribers_change'] = merge_df_users_fin['monthly_subscribers_gained'] - merge_df_users_fin['monthly_subscribers_lost']
merge_df_users_fin['quarterly_net_subscribers_change'] = merge_df_users_fin['quarterly_subscribers_gained'] - merge_df_users_fin['quarterly_subscribers_lost']

# 수익 관련 변수
merge_df_users_fin['weekly_estimated_revenue'] = merge_df_users_fin['estimated_revenue'].rolling(window=7).sum()
merge_df_users_fin['monthly_estimated_revenue'] = merge_df_users_fin['estimated_revenue'].rolling(window=30).sum()
merge_df_users_fin['quarterly_estimated_revenue'] = merge_df_users_fin['estimated_revenue'].rolling(window=90).sum()

merge_df_users_fin['weekly_estimated_ad_revenue'] = merge_df_users_fin['estimated_ad_revenue'].rolling(window=7).sum()
merge_df_users_fin['monthly_estimated_ad_revenue'] = merge_df_users_fin['estimated_ad_revenue'].rolling(window=30).sum()
merge_df_users_fin['quarterly_estimated_ad_revenue'] = merge_df_users_fin['estimated_ad_revenue'].rolling(window=90).sum()

merge_df_users_fin['weekly_revenue_per_subscriber'] = merge_df_users_fin['weekly_estimated_revenue'] / (merge_df_users_fin['weekly_subscribers_gained'] + 1)
merge_df_users_fin['monthly_revenue_per_subscriber'] = merge_df_users_fin['monthly_estimated_revenue'] / (merge_df_users_fin['monthly_subscribers_gained'] + 1)
merge_df_users_fin['quarterly_revenue_per_subscriber'] = merge_df_users_fin['quarterly_estimated_revenue'] / (merge_df_users_fin['quarterly_subscribers_gained'] + 1)

# 시청 시간 관련 변수
merge_df_users_fin['weekly_avg_view_duration'] = merge_df_users_fin['averageViewDuration'].rolling(window=7).mean()
merge_df_users_fin['monthly_avg_view_duration'] = merge_df_users_fin['averageViewDuration'].rolling(window=30).mean()
merge_df_users_fin['quarterly_avg_view_duration'] = merge_df_users_fin['averageViewDuration'].rolling(window=90).mean()

merge_df_users_fin['weekly_total_view_time'] = merge_df_users_fin['estimatedMinutesWatched'].rolling(window=7).sum()
merge_df_users_fin['monthly_total_view_time'] = merge_df_users_fin['estimatedMinutesWatched'].rolling(window=30).sum()
merge_df_users_fin['quarterly_total_view_time'] = merge_df_users_fin['estimatedMinutesWatched'].rolling(window=90).sum()

merge_df_users_fin['weekly_view_time_per_user'] = merge_df_users_fin['weekly_total_view_time'] / (merge_df_users_fin['weekly_subscribers_gained'] + 1)
merge_df_users_fin['monthly_view_time_per_user'] = merge_df_users_fin['monthly_total_view_time'] / (merge_df_users_fin['monthly_subscribers_gained'] + 1)
merge_df_users_fin['quarterly_view_time_per_user'] = merge_df_users_fin['quarterly_total_view_time'] / (merge_df_users_fin['quarterly_subscribers_gained'] + 1)

# 광고 관련 변수
merge_df_users_fin['weekly_playback_rate'] = merge_df_users_fin['playback_based_cpm'].rolling(window=7).mean()
merge_df_users_fin['monthly_playback_rate'] = merge_df_users_fin['playback_based_cpm'].rolling(window=30).mean()
merge_df_users_fin['quarterly_playback_rate'] = merge_df_users_fin['playback_based_cpm'].rolling(window=90).mean()

merge_df_users_fin['weekly_ad_revenue_rate'] = merge_df_users_fin['estimated_ad_revenue'] / (merge_df_users_fin['weekly_total_view_time'] + 1)
merge_df_users_fin['monthly_ad_revenue_rate'] = merge_df_users_fin['estimated_ad_revenue'] / (merge_df_users_fin['monthly_total_view_time'] + 1)
merge_df_users_fin['quarterly_ad_revenue_rate'] = merge_df_users_fin['estimated_ad_revenue'] / (merge_df_users_fin['quarterly_total_view_time'] + 1)

merge_df_users_fin['weekly_revenue_per_ad_impression'] = merge_df_users_fin['weekly_estimated_ad_revenue'] / (merge_df_users_fin['ad_impressions'].rolling(window=7).sum() + 1)
merge_df_users_fin['monthly_revenue_per_ad_impression'] = merge_df_users_fin['monthly_estimated_ad_revenue'] / (merge_df_users_fin['ad_impressions'].rolling(window=30).sum() + 1)
merge_df_users_fin['quarterly_revenue_per_ad_impression'] = merge_df_users_fin['quarterly_estimated_ad_revenue'] / (merge_df_users_fin['ad_impressions'].rolling(window=90).sum() + 1)

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [6]:
# 컬럼 정리
unique_col = merge_df_users_fin.columns[:11]
x_col = merge_df_users_fin.columns[11:].drop('subscribers_count') ## y값 제거

In [7]:
# 데이터 분할
train_data = merge_df_users_fin[merge_df_users_fin['date'] <= '2024-02-11']
test_data = merge_df_users_fin[merge_df_users_fin['date'] > '2024-02-11']

In [8]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(84473, 124)
(21210, 124)


### 변수선택

상관분석

In [13]:
# 상관계수 절대값이 0.3 이상
corr_df = train_data[['subscribers_count'] + list(x_col)].corr()
selected_features_by_corr = list(corr_df[corr_df['subscribers_count'] >= 0.3].iloc[:,0].keys()) + list(corr_df[corr_df['subscribers_count'] <= -0.3].iloc[:,0].keys())
selected_features_by_corr.remove('subscribers_count')
print(selected_features_by_corr)
print(len(selected_features_by_corr))

['redViews', 'SUBSCRIBED', 'estimatedMinutesWatched', 'estimated_revenue', 'estimated_ad_revenue', 'estimated_red_partner_revenue', 'gross_revenue', 'cpm', 'subscribers_lost', 'monetized_playbacks', 'revenue_per_ad_impression', 'watched_time_rate', 'gross_revenue_per_ad_impression', 'playback_rate', 'monthly_subscribers_gained', 'quarterly_subscribers_gained', 'weekly_subscribers_lost', 'monthly_subscribers_lost', 'quarterly_subscribers_lost', 'weekly_estimated_revenue', 'monthly_estimated_revenue', 'quarterly_estimated_revenue', 'weekly_estimated_ad_revenue', 'monthly_estimated_ad_revenue', 'quarterly_estimated_ad_revenue', 'weekly_total_view_time', 'monthly_total_view_time', 'quarterly_total_view_time', 'quarterly_revenue_per_ad_impression', 'unplayback_rate']
30


RandomForest

In [14]:
from sklearn.ensemble import RandomForestRegressor

# 랜덤 포레스트 모델 학습
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(train_data[x_col], train_data['subscribers_count'])

# 변수 중요도 추출
importances = rf_model.feature_importances_
feature_importance_rf = pd.Series(importances, index=x_col).sort_values(ascending=False)

# 중요도 0.001 이상 변수 선정
selected_features_by_rf = list(feature_importance_rf[feature_importance_rf >= 0.001].keys())

# 중요한 변수 출력
print(f"랜덤 포레스트로 선정된 변수: \n{selected_features_by_rf}")

Gradient Boosting

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting 모델 학습
gbm_model = GradientBoostingRegressor(random_state=42)
gbm_model.fit(train_data[x_col], train_data['subscribers_count'])

# 변수 중요도 추출
importances = gbm_model.feature_importances_
feature_importance_gbm = pd.Series(importances, index=x_col).sort_values(ascending=False)

# 중요도 0.001 이상 변수 선정
selected_features_by_gbm = list(feature_importance_gbm[feature_importance_gbm >= 0.001].keys())

# 중요한 변수 출력
print(f"Gradient Boosting으로 선정된 변수: \n{selected_features_by_gbm}")

Gradient Boosting으로 선정된 상위 10개 변수: 
subscribers_lost                   0.489201
subscriber_decrease_rate           0.144005
gross_revenue_per_ad_impression    0.126283
unplayback_rate                    0.074181
net_subscribers_change             0.045006
SUBSCRIBED                         0.032577
cpm                                0.031536
age13-17.female                    0.020211
revenue_per_subscriber             0.005551
subscriber_view_time_rate          0.004379
dtype: float64


LightGBM

In [ ]:
import lightgbm as lgb

# LightGBM 모델 학습
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(train_data[x_col], train_data['subscribers_count'])

# 변수 중요도 추출
importances = lgb_model.feature_importances_
feature_importance_lgbm = pd.Series(importances, index=x_col).sort_values(ascending=False)

# 중요도 0.001 이상 변수 선정
feature_importance_lgbm = feature_importance_lgbm / feature_importance_lgbm.sum() ## 중요도 정규화
selected_features_by_lgbm = list(feature_importance_lgbm[feature_importance_lgbm >= 0.005].keys())

# 중요한 변수 출력
print(f"LightGBM으로 선정된 변수: \n{selected_features_by_lgbm}")


c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018887 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19281
[LightGBM] [Info] Number of data points in the train set: 84473, number of used features: 76
[LightGBM] [Info] Start training from score 65433.322872
LightGBM으로 선정된 변수: 
['subscriber_decrease_rate', 'subscribers_lost', 'subscriber_increase_rate', 'net_subscribers_change', 'subscriber_view_time_rate', 'revenue_per_subscriber', 'gross_revenue_per_ad_impression', 'age18-24.female', 'subscribers_gained', 'dislikes', 'avg_view_duration_rate', 'estimated_red_partner_revenue', 'subscriber_retention_rate', 'comments', 'age13-17.female', 'age45-54.female', 'cpm', 'SUBSCRIBED', 'playback_based_cpm', 'subscribers_conversion_rate', 'age25-34.female', 'UNSUBSCRIBED', 'subscribed_view_time_rate', 'redViews', 'age45-54.male', 'like_to_dislike_r

XGBoost

In [26]:
import xgboost as xgb

# XGBoost 모델 학습
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(train_data[x_col], train_data['subscribers_count'])

# 변수 중요도 추출
importances = xgb_model.feature_importances_
feature_importance_xgb = pd.Series(importances, index=x_col).sort_values(ascending=False)

# 중요도 0.001 이상 변수 선정
selected_features_by_xgb = list(feature_importance_xgb[feature_importance_xgb >= 0.001].keys())

# 중요한 변수 출력
print(f"XGBoost로 선정된 변수: \n{selected_features_by_xgb}")

XGBoost로 선정된 상위 10개 변수: 
cpm                         0.572726
age13-17.female             0.139477
subscribers_lost            0.076834
net_subscribers_change      0.040480
adult_viewer_rate           0.024873
subscriber_decrease_rate    0.023228
age18-24.female             0.016084
age18-24.male               0.013767
SUBSCRIBED                  0.013691
age45-54.male               0.007620
dtype: float32


In [124]:
# 비선형모델 활용 변수 선정
importances_df = pd.DataFrame({
    'features': x_col,
    'rf_importance': rf_model.feature_importances_,
    'gbm_importance': gbm_model.feature_importances_,
    'lgbm_importance': lgb_model.feature_importances_,
    'xgb_importance': xgb_model.feature_importances_    
})
importances_df['lgbm_importance'] = importances_df['lgbm_importance'] / importances_df['lgbm_importance'].sum() ## 다른 모델과 중요도 단위 다르기 때문에 정규화
importances_df['mean_importance'] = importances_df[['rf_importance', 'gbm_importance', 'lgbm_importance', 'xgb_importance']].mean(axis=1) ## 각 변수별 모델 중요도 평균

In [133]:
# 최종 변수 선택
final_selected_features = list(set(list(importances_df[importances_df['mean_importance'] >= 0.004]['features']) + selected_features_by_corr))

In [138]:
# 최종 모델링 데이터셋
merge_df_users_final = merge_df_users_fin[list(unique_col) + final_selected_features + ['subscribers_count']]

In [143]:
# merge_df_users_final.to_csv('C:/py_src/awake/data/merge_df_users_final2.csv', encoding='utf-8-sig', index=False)

## 콘텐츠 데이터 분석

### 데이터 분할

In [73]:
# 컬럼 정리
unique_col = youtube_videos.columns[:3]
x_col = youtube_videos.columns[3:-1]

In [77]:
# 데이터 분할
X = youtube_videos[x_col].drop(columns=['total_engage_rate', 'net_subscribers_change', 'averageViewPercentage', 'revenue_per_view', 'grossRevenue_per_ad_impression','total_card_teaser_click_rate','playlist_engagement_rate']) ## y값 라벨링에 쓰인 지표 제거
y = youtube_videos['y_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# del X, y

In [78]:
# 불균형 확인
print(y_train.value_counts())
print(y_test.value_counts())

y_label
1    6513105
0     342503
Name: count, dtype: int64
y_label
1    1627933
0      85970
Name: count, dtype: int64


### 언더샘플링

In [79]:
# 언더샘플링
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

### 변수선택

t-test

In [80]:
from scipy.stats import ttest_ind

# y=0과 y=1 그룹으로 데이터 나누기
group_0 = X_train_resampled[y_train_resampled == 0]
group_1 = X_train_resampled[y_train_resampled == 1]

# t-test 수행
p_values = []
for col in X_train_resampled.columns:
    t_stat, p_val = ttest_ind(group_0[col], group_1[col], equal_var=False)  # Welch's t-test
    p_values.append((col, p_val))

# p-value < 0.05인 변수 선택
selected_features_by_ttest = [col for col, p_val in p_values if p_val < 0.05]
print("Selected Features by t-test:", selected_features_by_ttest)

Selected Features by t-test: ['views', 'redViews', 'comments', 'likes', 'dislikes', 'shares', 'estimatedMinutesWatched', 'estimatedRedMinutesWatched', 'averageViewDuration', 'videosAddedToPlaylists', 'videosRemovedFromPlaylists', 'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue', 'estimatedRedPartnerRevenue', 'playbackBasedCpm', 'cpm', 'subscribersGained', 'subscribersLost', 'monetizedPlaybacks', 'adImpressions', 'cardClickRate', 'cardTeaserClickRate', 'cardImpressions', 'cardTeaserImpressions', 'cardClicks', 'cardTeaserClicks', 'like_rate', 'comment_rate', 'share_rate', 'dislike_rate', 'positive_engage_rate', 'comment_to_like_rate', 'like_to_dislike_ratio', 'subscribers_conversion_rate', 'subscribers_gained_per_card_click', 'subscribers_gained_per_playlist_add', 'card_click_to_subscriber_conversion', 'subscribers_lost_per_playlist_remove', 'revenue_per_red_view', 'ad_revenue_rate', 'red_revenue_rate', 'cpm_to_revenue_ratio', 'revenue_per_ad_impression', 'playback_based_cpm_rate

In [81]:
len(selected_features_by_ttest)

72

Lasso

In [86]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# t-test로 선택된 변수로 데이터프레임 구성
X_train_ttest_selected = X_train_resampled[selected_features_by_ttest]

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ttest_selected)

# LassoCV 모델 설정: 반복 횟수 증가, alpha 범위 조정
lasso = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=[0.1, 0.05, 0.01, 0.005, 0.001]).fit(X_train_scaled, y_train_resampled)

# 선택된 변수 확인 (회귀 계수가 0이 아닌 변수들)
selected_features_by_lasso = X_train_ttest_selected.columns[(lasso.coef_ != 0)]
print("Selected Features by Lasso:", selected_features_by_lasso)

Selected Features by Lasso: Index(['views', 'redViews', 'comments', 'likes', 'dislikes', 'shares',
       'estimatedRedMinutesWatched', 'averageViewDuration',
       'videosAddedToPlaylists', 'videosRemovedFromPlaylists',
       'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue',
       'playbackBasedCpm', 'cpm', 'subscribersGained', 'subscribersLost',
       'monetizedPlaybacks', 'adImpressions', 'cardClickRate',
       'cardTeaserClickRate', 'cardImpressions', 'cardTeaserImpressions',
       'like_rate', 'comment_rate', 'dislike_rate', 'positive_engage_rate',
       'comment_to_like_rate', 'like_to_dislike_ratio',
       'subscribers_conversion_rate', 'subscribers_gained_per_playlist_add',
       'revenue_per_red_view', 'ad_revenue_rate', 'red_revenue_rate',
       'cpm_to_revenue_ratio', 'revenue_per_ad_impression',
       'playback_based_cpm_rate', 'revenue_per_playlist_add',
       'net_revenue_per_playlist_add', 'revenue_per_minute_watched',
       'avg_view_duration_rate',

In [87]:
len(selected_features_by_lasso)

54

In [91]:
## lasso 선택으로 제거된 변수
set(selected_features_by_ttest) - set(selected_features_by_lasso)

{'ad_impressions_per_card_click',
 'ad_impressions_per_playlist_add',
 'ad_playbacks_per_card_click',
 'ad_revenue_per_card_click',
 'cardClicks',
 'cardTeaserClicks',
 'card_click_to_subscriber_conversion',
 'card_to_teaser_click_rate',
 'estimatedMinutesWatched',
 'estimatedRedPartnerRevenue',
 'net_playlist_addition_rate',
 'playback_rate',
 'revenue_per_card_click',
 'share_rate',
 'subscribers_gained_per_card_click',
 'subscribers_lost_per_playlist_remove',
 'watch_time_per_card_click',
 'watched_time_rate'}

RandomForest

In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# RandomForest 모델 학습 및 교차 검증
X_train_lasso_selected = X_train_resampled[selected_features_by_lasso]

model = RandomForestClassifier(random_state=42)
cross_val_scores = cross_val_score(model, X_train_lasso_selected, y_train_resampled, cv=5, scoring='accuracy')

# 모델 학습
model.fit(X_train_lasso_selected, y_train_resampled)

# 피처 중요도 추출
importances = model.feature_importances_
feature_importances = pd.DataFrame({'feature': selected_features_by_lasso, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# 최종적으로 중요한 변수 선택 (예: 상위 10개 변수)
final_selected_features = list(feature_importances[feature_importances['importance'] > 0.005]['feature'])
print("Final Selected Features by RandomForest:", final_selected_features)

# 교차 검증 결과 출력
print(f"Cross-Validation Accuracy: {cross_val_scores.mean():.2f}")

Final Selected Features by RandomForest: ['positive_engage_rate', 'playlist_addition_rate', 'cpm', 'revenue_per_ad_impression', 'like_rate', 'revenue_per_playback', 'playlist_removal_rate', 'videosAddedToPlaylists', 'videosRemovedFromPlaylists', 'views', 'likes', 'subscribersGained', 'watch_time_per_playlist_add', 'playbackBasedCpm', 'revenue_per_red_view', 'averageViewDuration', 'shares', 'estimatedRevenue', 'avg_view_duration_rate', 'redViews', 'watch_time_loss_per_playlist_remove', 'ad_revenue_rate', 'playlist_related_revenue_rate', 'estimatedRedMinutesWatched', 'watched_view_rate', 'grossRevenue', 'revenue_per_minute_watched', 'comment_rate', 'net_revenue_per_playlist_add', 'cpm_to_revenue_ratio', 'ad_playbacks_per_playlist_add', 'adImpressions', 'estimatedAdRevenue', 'comments', 'subscribers_conversion_rate', 'monetizedPlaybacks', 'dislikes']
Cross-Validation Accuracy: 0.99


In [102]:
len(final_selected_features)

37

In [94]:
feature_importances ## 0.005 기준으로 변수 선택

,feature,importance
26,positive_engage_rate,0.111536
51,playlist_addition_rate,0.079711
14,cpm,0.077948
35,revenue_per_ad_impression,0.064829
23,like_rate,0.064504
45,revenue_per_playback,0.054518
52,playlist_removal_rate,0.045625
8,videosAddedToPlaylists,0.038797
9,videosRemovedFromPlaylists,0.034812
0,views,0.033525


In [100]:
youtube_videos_final = youtube_videos[list(unique_col) + final_selected_features + ['y_label']]

In [101]:
# youtube_videos_final.to_csv('C:/py_src/awake/data/youtube_videos_final.csv', encoding='utf-8-sig', index=False)